In [1]:
import pandas as pd
import sys
import os
import numpy 
import pylab as pl
import geopandas as gp

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
nrg = gp.GeoDataFrame.from_csv('C:/Users/Maria/SkyDrive/Documents/principles of informatics/Energy_and_Water_Data_disclosure_for_Local_Law_84__2013_.csv')
print nrg.columns

Index([u'NYC Borough, Block, and Lot (BBL)', u'Co-reported BBL Status',
       u'BBLs Co-reported',
       u'Reported NYC Building Identificaiton Numbers (BINs)',
       u'Street Number', u'Street Name', u'Borough', u'Zip Code',
       u'DOF Benchmarking Submission Status', u'Site EUI(kBtu/ft2)',
       u'Weather Normalized Site EUI(kBtu/ft2)', u'Source EUI(kBtu/ft2)',
       u'Weather Normalized Source EUI(kBtu/ft2)',
       u'Municipally Supplied Potable Water - Indoor Intensity (gal/ft²)',
       u'Automatic Water Benchmarking Eligible', u'Reported Water Method',
       u'ENERGY STAR Score', u'Total GHG Emissions(MtCO2e)',
       u'Direct GHG Emissions(MtCO2e)', u'Indirect GHG Emissions(MtCO2e)',
       u'Reported Property Floor Area (Building(s)) (ft²)',
       u'DOF Property Floor Area (Buildngs and Parking)(ft2)',
       u'Primary Property Type - Self Selected', u'DOF Number of Buildings'],
      dtype='object')


In [ ]:
bsize = gp.GeoDataFrame.from_file('C:/Users/Maria/SkyDrive/Documents/principles of informatics/Manhattan/MNMapPLUTO.shp')
print bsize.columns

In [ ]:
from pandas.tools.plotting import scatter_matrix
scatter_matrix (nrg, s=300, figsize=(16, 16), diagonal='kde')
# scatter matrix only shows non-Nan values so not much use at this time

In [ ]:
# create the indeces array

nrg['Reported Property Floor Area (Building(s)) (ft²)']=nrg['Reported Property Floor Area (Building(s)) (ft²)'].convert_objects(convert_numeric=True)
nrg['Site EUI(kBtu/ft2)']=nrg['Site EUI(kBtu/ft2)'].convert_objects(convert_numeric=True)

# print for clarification (if needed)
#print nrg['Site EUI(kBtu/ft2)'].astype(float)
#print bsize.BBL.astype(float)


In [ ]:
# merge datasets using newly created created column BBL
nrg['BBL'] = nrg['NYC Borough, Block, and Lot (BBL)']
bblnrgdata = nrg.merge(bsize, on='BBL')

In [ ]:
bblnrgdata.head()

In [ ]:
# plot energy total per building by number of units in building
datay = (bblnrgdata['Site EUI(kBtu/ft2)'])*(bblnrgdata['Reported Property Floor Area (Building(s)) (ft²)']) # total energy
datax = bblnrgdata['UnitsTotal']

In [ ]:
fig=figure(figsize = (10,10))

ax = fig.add_subplot(111)
ax.plot(datax, datay, 'bo')
ax.set_xlim(1,3000)
ax.set_ylim(1000, 1e11)
plt.xlabel('number of total units')
plt.ylabel('total energy per building')

In [ ]:
# cut dataset to remove zeros and other leverage points

index = (datax>0)&(datax<4000)&(datay>1000)&(datay<1e11) # must use apersand, will assign true or false
#print(index)


In [ ]:
# run log regression
import statsmodels.api as sm
X = sm.add_constant(np.log10(datax[index])) # add an intercept
#print X
model1 = sm.OLS(np.log10(datay[index]), X, missing='drop').fit()
print model1.summary()

In [ ]:
#plot model results above
Y = np.log10(datay[index])

fig2 = figure(figsize = (10,10))
XLR = np.arange(0, 4, 1)
YLR = 0.0822*XLR + 6.9178
ax = fig2.add_subplot(111)
ax.plot(X, Y, 'ro')
ax.plot(XLR, YLR, 'blue')
plt.xlabel('number of total units - log value')
plt.ylabel('total energy per building - log value')



Cannot perform log likelihood ratio test because degrees of freedom are needed are needed for chi-squared distribution (how D is distributed) and it's zero because there is only on regressor in each model.

In [ ]:
X2 = sm.add_constant(np.log10(datay[index]))
model2 = sm.OLS(np.log10(datax[index]), X2, missing='drop').fit()
print model2.summary()


In [ ]:
# flip x and y axis to see if fit is "better"
X2 = np.log10(datay[index])
Y2 = sm.add_constant(np.log10(datax[index]))
fig3 = figure(figsize = (10,10))
XLR = np.arange(1, 10, 1)
YLR = 0.1418*XLR + 0.662
ax = fig3.add_subplot(111)
ax.plot(X2, Y2, 'ro')
ax.plot(XLR, YLR, 'blue')
plt.ylabel('number of total units - log value')
plt.ylabel('total energy per building - log value')

In [ ]:
# second degree curve

model2 = np.polyfit(datax[index], datay[index], 2)

dataIX = np.log10(datax[index])
dataIY = np.log10(datay[index])

model3 = np.polyfit(dataIX, dataIY, 2)

print model2
print model3

In [ ]:
#fit second degree curve
# use same scatter plot as logTotalUnits, logTotalEnergyPerBldg
fig4 = figure(figsize = (10,10))
ax = fig4.add_subplot(111)
ax.plot (dataIX, dataIY, 'go')
XLR = np.arange(1, 5, .2)
XLY = 0.353*XLR**2 -0.782*XLR + 7.3072
ax.plot (XLR, XLY, 'black')
plt.title ('Fitting a 2nd degree Curve')
plt.xlabel('number of total units - log value')
plt.ylabel('total energy per building - log value')



may be able to perfom perform log-likelihood ratio becasue degrees of freedom between the model is one and the chi-squared stat requires df